# ATLAS Multihop QA Benchmarking

In [2]:
# 打印Python版本
import sys
print("Python version:", sys.version)

Python version: 3.10.18 (main, Jun  5 2025, 08:37:47) [Clang 14.0.6 ]


In [3]:
# 设置使用CPU模式
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = ''  # 禁用GPU，强制使用CPU
import torch
print("使用设备: CPU")
print("CUDA可用:", torch.cuda.is_available())

使用设备: CPU
CUDA可用: False


In [4]:
from atlas_rag.vectorstore.embedding_model import SentenceEmbedding
from sentence_transformers import SentenceTransformer

# 加载SentenceTransformer模型 (CPU版本)
encoder_model_name = 'all-MiniLM-L6-v2'
sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
sentence_encoder = SentenceEmbedding(sentence_model)

print(f"使用编码器模型: {encoder_model_name}")
print(f"模型设备: {sentence_model.device}")

使用编码器模型: all-MiniLM-L6-v2
模型设备: cpu


In [10]:
# 使用DeepInfra的Meta-Llama模型
from openai import OpenAI
from atlas_rag.llm_generator import LLMGenerator
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')

# 设置DeepInfra客户端
reader_model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # 使用与提取时相同的模型
client = OpenAI(
    base_url="https://api.deepinfra.com/v1/openai",
    api_key=config['settings']['DEEPINFRA_API_KEY']
)

llm_generator = LLMGenerator(client=client, model_name=reader_model_name)
print(f"使用LLM模型: {reader_model_name}")
print("API端点: DeepInfra")

使用LLM模型: meta-llama/Meta-Llama-3.1-8B-Instruct
API端点: DeepInfra


In [6]:
from openai import OpenAI
from atlas_rag.llm_generator import LLMGenerator
from configparser import ConfigParser
# Load OpenRouter API key from config file
config = ConfigParser()
config.read('config.ini')
reader_model_name = "meta-llama/Llama-3.3-70B-Instruct"
client = OpenAI(
  base_url="https://api.deepinfra.com/v1/openai",
  api_key=config['settings']['DEEPINFRA_API_KEY'],
)
llm_generator = LLMGenerator(client=client, model_name=reader_model_name)


In [ ]:
from atlas_rag.vectorstore import create_embeddings_and_index

# 使用与提取时相同的设置
keyword = 'musique_sample'  # 使用样本数据集的结果
working_directory = './working_data'

print(f"加载数据集: {keyword}")
print(f"工作目录: {working_directory}")

data = create_embeddings_and_index(
    sentence_encoder=sentence_encoder,
    model_name=encoder_model_name,
    working_directory=working_directory,
    keyword=keyword,
    include_concept=True,
    include_events=True,
    normalize_embeddings=True,
    text_batch_size=16,  # CPU版本使用较小的batch size
    node_and_edge_batch_size=16,
)

加载数据集: musique_sample
工作目录: ./working_data
Using encoder model: all-MiniLM-L6-v2
Loading graph from ./working_data/kg_graphml/musique_sample_graph.graphml


100%|██████████| 18717/18717 [00:00<00:00, 2486374.48it/s]
87690it [00:00, 5748917.86it/s]


Text embeddings already computed.
Node and edge embeddings not found, computing...


Encoding edges: 100%|██████████| 4674/4674 [02:11<00:00, 35.46it/s]

In [1]:
from atlas_rag.evaluation import BenchMarkConfig
benchmark_config = BenchMarkConfig(
    dataset_name= 'musique',
    question_file= "benchmark_data/musique.json",
    include_concept=True,
    include_events=True,
    reader_model_name=reader_model_name,
    encoder_model_name=encoder_model_name,
    number_of_samples=5, # -1 for all samples
)

NameError: name 'reader_model_name' is not defined

In [6]:
from atlas_rag import setup_logger
logger = setup_logger(benchmark_config)

In [ ]:
# Initialize desired RAG method for benchmarking
from atlas_rag.retriever import HippoRAG2Retriever
hipporag2_retriever = HippoRAG2Retriever(
    llm_generator=llm_generator,
    sentence_encoder=sentence_encoder,
    data = data,
    logger=logger
)

In [ ]:
# start benchmarking
from atlas_rag.evaluation import RAGBenchmark
benchmark = RAGBenchmark(config=benchmark_config, logger=logger)
benchmark.run([hipporag2_retriever], llm_generator=llm_generator) 
# benchmark.run([hipporag2_retriever], llm_generator=llm_generator, use_react=True) # use_react=True to enable RAG with REACT